<a href="https://colab.research.google.com/github/andyarnell/sepal_mgci/blob/master/Calculation_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
#removing folders if needed from colab
# import shutil
# import os
# shutil.rmtree('')#, ignore_errors=True)


#### Clone SDG 15.4.2 github repo into colab instance

In [22]:
%cd "/content"
!git clone https://github.com/andyarnell/sepal_mgci.git

/content
fatal: destination path 'sepal_mgci' already exists and is not an empty directory.


In [23]:
import ee
def install_if_not_exists(package_name):
    """
    Installs a Python package if it's not already installed.

    Parameters:
    package_name (str): The name of the package to install.
    """
    try:
        import package_name
        print(f"{package_name} is already installed.")
    except ImportError:
        !pip install {package_name}
        print(f"{package_name} has been installed.")


install_if_not_exists('pygaul')
install_if_not_exists('geemap')
install_if_not_exists('unidecode')
install_if_not_exists('sepal_ui')


import pygaul
import pandas as pd
from unidecode import unidecode
import geemap
import re

from pathlib import Path

from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

%cd "/content/sepal_mgci"

# import component.parameter.module_parameter as param ## using only parameters needed instead

from component.parameter.module_parameter import DEM_DEFAULT
from component.parameter.module_parameter import LC_MAP_MATRIX

gee_project_name = "ee-andyarnellgee" # need a valid google project name
ee.Authenticate()
ee.Initialize(project = gee_project_name)

from component.scripts.gee import reduce_regions
from component.scripts.scripts import get_a_years, map_matrix_to_dict, parse_result
from component.scripts.scripts import read_from_csv
from component.scripts import sub_a as sub_a
from component.scripts import sub_b as sub_b
from component.scripts import mountain_area as mntn

print ("imports complete")

###following may not be needed - for reading google drive
# Install the necessary libraries
!pip install -U -q google-api-python-client google-auth-httplib2 google-auth-oauthlib

from google.colab import auth
from googleapiclient.discovery import build

pygaul has been installed.
geemap has been installed.
unidecode has been installed.
sepal_ui has been installed.
/content/sepal_mgci
imports complete


#### Access to Google Drive for script to retrieve output csvs.

Alternatively these can be uploaded. To do open the File panel by clicking on the file icon on the left, then navigate to folder XXXX and click the upload icon (a sheet of paper with an up arrow in it) to upload the files (may need to zip, upload then unzip).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [25]:


def folder_exists(folder_name, parent_folder_id=None):
    """
    Check if a folder exists in Google Drive.

    Args:
    - folder_name (str): Name of the folder to check.
    - parent_folder_id (str): ID of the parent folder where to search for the folder.
                              Default is None, meaning the search will be performed in the root.

    Returns:
    - bool: True if the folder exists, False otherwise.
    """
    # Authenticate user
    auth.authenticate_user()

    # Build the Drive v3 service
    drive_service = build('drive', 'v3')

    # Prepare query to check if folder exists
    query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
    if parent_folder_id:
        query += f" and '{parent_folder_id}' in parents"

    try:
        # Execute the search query
        folders = drive_service.files().list(q=query, fields='files(id)', includeItemsFromAllDrives=True, supportsAllDrives=True).execute().get('files', [])
        return bool(folders)
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

# # Example usage:
# folder_name = "MyFolder"
# parent_folder_id = None  # Change this to the parent folder ID if necessary
# exists = folder_exists(folder_name, parent_folder_id)
# print(f"Folder '{folder_name}' exists: {exists}")


def create_folder(folder_name, parent_folder_id=None):
    """
    Create a folder in Google Drive.

    Args:
    - folder_name (str): Name of the folder to be created.
    - parent_folder_id (str): ID of the parent folder where the new folder will be created.
                              Default is None, meaning the folder will be created in the root.

    Returns:
    - str: ID of the newly created folder.
    """
    # Authenticate user
    auth.authenticate_user()

    # Build the Drive v3 service
    drive_service = build('drive', 'v3')

    # Prepare folder metadata
    folder_metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    if parent_folder_id:
        folder_metadata['parents'] = [parent_folder_id]

    # Create the folder
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()

    # Return the ID of the newly created folder
    return folder.get('id')

def create_folder_if_not_exists(folder_name, parent_folder_id=None):
    """
    Create a folder in Google Drive if it doesn't already exist.

    Args:
    - folder_name (str): Name of the folder to be created.
    - parent_folder_id (str): ID of the parent folder where the new folder will be created.
                              Default is None, meaning the folder will be created in the root.

    Returns:
    - str: ID of the newly created folder or the existing folder if it already exists.
    """
    if folder_exists(folder_name, parent_folder_id):
        print(f"Folder '{folder_name}' already exists.")
        return None
    else:
        return create_folder(folder_name, parent_folder_id)


def sanitize_description(description):
    allowed_characters_pattern = r"[^a-zA-Z0-9.,:;_ \-]"  # Define a regex pattern for characters not in the allowed set
    sanitized_description = re.sub(allowed_characters_pattern, "", description)  # Remove characters not in the allowed set
    return sanitized_description
sanitize_description("bnikd ).")

####Parameters:
Set output folder paths etc

In [53]:
drive_home ="/content/drive/MyDrive/"
stats_csv_folder = "sdg_15_4_2_csvs"+"/"

report_folder = "sdg_15_4_2_reports"
report_file = "sdg_report_name.xlsx"

# Optional: If you want to place the folder inside another folder, provide the parent folder ID.
# If not, leave parent_folder_id as None.
parent_folder_id = None

# Create the folder in Google Drive if it doesn't exist
create_folder_if_not_exists(report_folder)#, parent_folder_id)

Folder 'sdg_15_4_2_reports' already exists.


In [55]:
admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.gt("Shape_Area",10)).limit(10)

## SUB INDICATOR A

In [58]:
# Read the default land cover remapping table and convert it to a dictionary
# default_map_matrix = map_matrix_to_dict(param.LC_MAP_MATRIX)
default_map_matrix = map_matrix_to_dict(LC_MAP_MATRIX)

In [59]:
# For SUB_A indicator, we need to set the following structure
a_years = {
    1: {"asset": "users/amitghosh/sdg_module/esa/cci_landcover/2000", "year": 2000},
    2: {"year": 2003, "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2003"},
    3: {"year": 2007, "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2007"},
    4: {"year": 2010, "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2010"},
}

# Just extract the years from the a_years dictionary
single_years = [y["year"] for  y in a_years.values()]

In [60]:
counter=0

for aoi_name in list_of_countries:

    aoi = admin_boundaries.filter(ee.Filter.eq(admin_property,aoi_name))#.first()

    process = ee.FeatureCollection([
        ee.Feature(
            None,
            reduce_regions(
                aoi,
                remap_matrix=default_map_matrix,
                rsa=False,
                # dem=param.DEM_DEFAULT,
                dem=DEM_DEFAULT,
                lc_years= year,
                transition_matrix=False
            )
        ).set("process_id", year[0]["year"])
        for year in get_a_years(a_years)
    ])

    #make name acceptable for running tasks (i.e., removes special characters)
    task_name = str(sanitize_description(unidecode(aoi_name)))


    task = ee.batch.Export.table.toDrive(
        **{  #asterisks unpack dictionary into keyword arguments format
            "collection": process,
            "description": task_name,
            "fileFormat": "CSV",
            "folder":stats_csv_folder,
            "selectors": [
                "process_id",
                "sub_a",
            ],
        }
    )

    counter+=1

    print ("process #",counter)

    print (aoi_name)

    task.start()



process # 1
309.22080775909325
309.22080775909325
309.22080775909325
309.22080775909325
South Sudan
process # 2
309.22080775909325
309.22080775909325
309.22080775909325
309.22080775909325
Sudan
process # 3
309.22080775909325
309.22080775909325
309.22080775909325
309.22080775909325
Greece
process # 4
309.22080775909325
309.22080775909325
309.22080775909325
309.22080775909325
Italy
process # 5
309.22080775909325
309.22080775909325
309.22080775909325
309.22080775909325
Spain
process # 6
309.22080775909325
309.22080775909325
309.22080775909325
309.22080775909325
France
process # 7
309.22080775909325
309.22080775909325
309.22080775909325
309.22080775909325
Germany
process # 8
309.22080775909325
309.22080775909325
309.22080775909325
309.22080775909325
Austria
process # 9
309.22080775909325
309.22080775909325
309.22080775909325
309.22080775909325
Australia
process # 10
309.22080775909325
309.22080775909325
309.22080775909325
309.22080775909325


KeyboardInterrupt: 

# Read, process, and create report tables

#####Manually check your earth engine task status, once the task is completed, run the next cell
##### according to the task name you have set above.

In [64]:

counter = 0

for aoi_name in list_of_countries:

  counter+=1

  print ("process #",counter)

  aoi_name_clean = str(sanitize_description(unidecode(aoi_name)))

  stats_csv_file = aoi_name_clean+".csv"
  stats_csv_file_path = drive_home + stats_csv_file

  print (stats_csv_file_path)
  # This line will read the results from the CSV file and parse it to a dictionary
  dict_results = read_from_csv(stats_csv_file_path)
  dict_results

  details = {
      "geo_area_name" : aoi_name,
      "ref_area" : " ",
      "source_detail" : " ",
  }
  # Following lines will generate the reports for the sub_a and mtn indicators
  sub_a_reports = []
  mtn_reports = []

  for year in single_years:
      print(f"Reporting {year} for sub_a")
      parsed_df = parse_result(dict_results[year]["sub_a"], single=True)

      sub_a_reports.append(sub_a.get_reports(parsed_df, year, **details))

      print(f"Reporting {year} for mtn")
      mtn_reports.append(mntn.get_report(parsed_df, year, **details))

  mtn_reports_df = pd.concat(mtn_reports)

  # sub a reports
  er_mtn_grnvi_df = pd.concat([report[0] for report in sub_a_reports])
  er_mtn_grncov_df = pd.concat([report[1] for report in sub_a_reports])

  # define where to put output reports
  report_file_path = drive_home + report_folder + "/"+aoi_name_clean +".xlsx"

  print (report_file_path)

  # This will create the excel file with the reports
  with pd.ExcelWriter(report_file_path) as writer:
      mtn_reports_df.to_excel(writer, sheet_name="Table1_ER_MTN_TOTL", index=False)
      er_mtn_grncov_df.to_excel(
          writer, sheet_name="Table2_ER_MTN_GRNCOV", index=False
      )
      er_mtn_grnvi_df.to_excel(writer, sheet_name="Table3_ER_MTN_GRNCVI", index=False)

      for sheetname in writer.sheets:
          worksheet = writer.sheets[sheetname]
          for col in worksheet.columns:
              max_length = 0
              column = col[0]
              for cell in col:
                  try:
                      if len(str(cell.value)) > max_length:
                          max_length = len(cell.value)
                  except:
                      pass
              adjusted_width = max(max_length, len(str(column.value))) + 4
              worksheet.column_dimensions[get_column_letter(column.column)].width = (
                  adjusted_width
              )

              # Align "obs_value" column to the right
              if "OBS" in column.value:
                  for cell in col:
                      cell.alignment = Alignment(horizontal="right")

process # 1
/content/drive/MyDrive/South Sudan.csv
Reporting 2000 for sub_a
Reporting 2000 for mtn
Reporting 2003 for sub_a
Reporting 2003 for mtn
Reporting 2007 for sub_a
Reporting 2007 for mtn
Reporting 2010 for sub_a
Reporting 2010 for mtn
/content/drive/MyDrive/sdg_15_4_2_reports/South Sudan.xlsx
process # 2
/content/drive/MyDrive/Sudan.csv
Reporting 2000 for sub_a
Reporting 2000 for mtn
Reporting 2003 for sub_a
Reporting 2003 for mtn
Reporting 2007 for sub_a
Reporting 2007 for mtn
Reporting 2010 for sub_a
Reporting 2010 for mtn
/content/drive/MyDrive/sdg_15_4_2_reports/Sudan.xlsx
process # 3
/content/drive/MyDrive/Greece.csv
Reporting 2000 for sub_a
Reporting 2000 for mtn
Reporting 2003 for sub_a
Reporting 2003 for mtn
Reporting 2007 for sub_a
Reporting 2007 for mtn
Reporting 2010 for sub_a
Reporting 2010 for mtn
/content/drive/MyDrive/sdg_15_4_2_reports/Greece.xlsx
process # 4
/content/drive/MyDrive/Italy.csv
Reporting 2000 for sub_a
Reporting 2000 for mtn
Reporting 2003 for sub_

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Australia.csv'

In [ ]:
# # Following lines will generate the reports for the sub_a and mtn indicators
# sub_a_reports = []
# mtn_reports = []

# for year in single_years:
#     print(f"Reporting {year} for sub_a")
#     parsed_df = parse_result(dict_results[year]["sub_a"], single=True)

#     sub_a_reports.append(sub_a.get_reports(parsed_df, year, **details))

#     print(f"Reporting {year} for mtn")
#     mtn_reports.append(mntn.get_report(parsed_df, year, **details))


In [ ]:
# mtn_reports_df = pd.concat(mtn_reports)

# # sub a reports
# er_mtn_grnvi_df = pd.concat([report[0] for report in sub_a_reports])
# er_mtn_grncov_df = pd.concat([report[1] for report in sub_a_reports])


In [ ]:
# This will create the excel file with the reports

with pd.ExcelWriter(output_name) as writer:
    mtn_reports_df.to_excel(writer, sheet_name="Table1_ER_MTN_TOTL", index=False)
    er_mtn_grncov_df.to_excel(
        writer, sheet_name="Table2_ER_MTN_GRNCOV", index=False
    )
    er_mtn_grnvi_df.to_excel(writer, sheet_name="Table3_ER_MTN_GRNCVI", index=False)

    for sheetname in writer.sheets:
        worksheet = writer.sheets[sheetname]
        for col in worksheet.columns:
            max_length = 0
            column = col[0]
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = max(max_length, len(str(column.value))) + 4
            worksheet.column_dimensions[get_column_letter(column.column)].width = (
                adjusted_width
            )

            # Align "obs_value" column to the right
            if "OBS" in column.value:
                for cell in col:
                    cell.alignment = Alignment(horizontal="right")